In [1]:
import os
import difflib
import collections
from datetime import datetime
import openpyxl

In [2]:
# enter basic informations like checkpoint_path and configs_path

ckpt_path = "/root/ext_data/studiogan_ckpt/AFHQv2_uncond_512/checkpoints"
cfg_path = "./src/configs/AFHQv2_uncond_512"
dataset_path = "/root/code/joong/data/AFHQv2_uncond_512"
dataset = "AFHQv2_uncond_512"
num_eval = 1

ckpt_list = os.listdir(ckpt_path)
cfg_list = os.listdir(cfg_path)

# load Excel File and read informations
wb = openpyxl.load_workbook("Taxonomy_experiments_offline.xlsx")
ws = wb['AFHQ-V2-512-uncond-tailored']

runs = collections.defaultdict(lambda : collections.defaultdict(str))

all_columns = ws.columns
for idx,col in enumerate(all_columns):
    if idx >=2 and col[1].value != None:
        assert col[6].value in ckpt_list, str(col[6].value) + " Not in ckpt_list"
        runs[col[1].value]["run_1"] = col[6].value
        if col[7].value != None:
            assert col[7].value in ckpt_list, str(col[7].value) + " Not in ckpt_list"
            runs[col[1].value]["run_2"] = col[7].value
        if col[8].value != None:
            assert col[8].value in ckpt_list, str(col[8].value) + " Not in ckpt_list"
            runs[col[1].value]["run_3"] = col[8].value
            
        if col[1].value == "ACGAN":
            runs[col[1].value]["cfg"] = difflib.get_close_matches(col[1].value+"-Mod", cfg_list, n=1, cutoff=0)[0]
        else:
            runs[col[1].value]["cfg"] = difflib.get_close_matches(col[1].value, cfg_list, n=1, cutoff=0)[0]
        print(f"{col[1].value} matched with {runs[col[1].value]['cfg']}")

FileNotFoundError: [Errno 2] No such file or directory: '/root/ext_data/studiogan_ckpt/AFHQv2_uncond_512/checkpoints'

In [ ]:
# Check if evey excel column is matched with proper configurations and modify config if needed.
# runs["TACGAN"]["cfg"] = "ACGAN-Mod-TAC.yaml"
# runs["CRGAN"]["cfg"] = "BigGAN-CR.yaml"
# runs["ICRGAN"]["cfg"] = "BigGAN-ICR.yaml"
# runs["ADCGAN"]["cfg"] = "ACGAN-Mod-Big-ADC.yaml"
# runs["BigGAN-LeCam"]["cfg"] = "BigGAN-256Res-LeCam.yaml"
# runs["StyleGAN2-LeCam"]["cfg"] = "StyleGAN2-256Res-LeCam.yaml"

print("After fixing...")
for key, value in runs.items():
    print(f"{key} matched with {value['cfg']}")

In [ ]:
# write script file
i = 1
with open("eval_"+dataset+".sh", "w") as f:
    f.write("wandb offline\n")
    train_ = " -ref train "
    metrics_full = " -metrics is fid prdc -ifid "
    metrics_fide = " -metrics is fid "
    eval_backbone_incep = " --eval_backbone InceptionV3_tf "
    eval_backbone_swav = " --eval_backbone SwAV_torch "
    eval_backbone_swin = " --eval_backbone Swin-T_torch "
    
    if dataset == "ImageNet":
        prefix_ = "CUDA_VISIBLE_DEVICES=0,1 python src/main.py --pre_resizer lanczos --post_resizer tailored -sync_bn -std_stat -std_max 256 -std_step 256 -best --seed 1234" + " -data " + dataset_path + " -save ../studiogan/" + " --num_eval " + str(num_eval)
        prefix_style = "CUDA_VISIBLE_DEVICES=0,1 python src/main.py --pre_resizer lanczos --post_resizer tailored -best --seed 1234 -mpc -DDP" + " -data " + dataset_path + " -save ../studiogan/" + " --num_eval " + str(num_eval)
        eval_ = " -ref valid "
    elif dataset in ["Baby_ImageNet", "Papa_ImageNet", "Grandpa_ImageNet"]:
        prefix_S = "CUDA_VISIBLE_DEVICES=0,1,2,3 python src/main.py --pre_resizer lanczos --post_resizer tailored -sync_bn -std_stat -std_max 256 -std_step 256 -best --seed 1234" + " -data " + dataset_path + " -save ../studiogan/" + " --num_eval " + str(num_eval)
        prefix_B = "CUDA_VISIBLE_DEVICES=0,1,2,3 python src/main.py --pre_resizer lanczos --post_resizer tailored -sync_bn -std_stat -std_max 1024 -std_step 1024 -best --seed 1234" + " -data " + dataset_path + " -save ../studiogan/" + " --num_eval " + str(num_eval)
        prefix_style = "CUDA_VISIBLE_DEVICES=0,1 python src/main.py --pre_resizer lanczos --post_resizer tailored -best --seed 1234 -mpc -DDP" + " -data " + dataset_path + " -save ../studiogan/" + " --num_eval " + str(num_eval)
        eval_ = " -ref valid "
    elif dataset ==  "AFHQv2":
        prefix_ = "CUDA_VISIBLE_DEVICES=0,1,2,3 python src/main.py --pre_resizer lanczos --post_resizer tailored -sync_bn -std_stat -std_max 256 -std_step 256 -best --seed 1234" + " -data " + dataset_path + " -save ../studiogan/" + " --num_eval " + str(num_eval)
        prefix_style = "CUDA_VISIBLE_DEVICES=0,1 python src/main.py --pre_resizer lanczos --post_resizer tailored -best --seed 1234 -mpc -DDP" + " -data " + dataset_path + " -save ../studiogan/" + " --num_eval " + str(num_eval)
        eval_ = " -ref valid "
    elif dataset == "AFHQv2_uncond_512":
        prefix_style = "CUDA_VISIBLE_DEVICES=0 python src/main.py --pre_resizer lanczos --post_resizer tailored -best --seed 1234 -mpc" + " -data " + dataset_path + " -save ../studiogan/" + " --num_eval " + str(num_eval)
    elif dataset.startswith("CIFAR10"):
        prefix_ = "CUDA_VISIBLE_DEVICES=0 python src/main.py --pre_resizer wo_resize --post_resizer tailored -std_stat -std_max 128 -std_step 128 -best --seed 1234" + " -data " + dataset_path + " -save ../studiogan/" + " --num_eval " + str(num_eval)
        prefix_style = "CUDA_VISIBLE_DEVICES=0 python src/main.py --pre_resizer wo_resize --post_resizer tailored -best --seed 1234 -mpc" + " -data " + dataset_path + " -save ../studiogan/" + " --num_eval " + str(num_eval)
        eval_ = " -ref test "
    
    for (key, value) in runs.items():
        others1 = " -ckpt " + os.path.join(ckpt_path, value['run_1']) + " -cfg " + os.path.join(cfg_path, value['cfg'])
        if value['run_2'] != '':
            others2 = " -ckpt " + os.path.join(ckpt_path, value['run_2']) + " -cfg " + os.path.join(cfg_path, value['cfg'])
        else:
            others2 = None
        if value['run_3'] != '':
            others3 = " -ckpt " + os.path.join(ckpt_path, value['run_3']) + " -cfg " + os.path.join(cfg_path, value['cfg'])
        else:
            others3 = None
        
        if "Style" in key:
            prefix = prefix_style
        elif dataset in ["Baby_ImageNet","Papa_ImageNet", "Grandpa_ImageNet"] and ("SAGAN" in key or "SNGAN" in key):
            prefix = prefix_S
        elif dataset in ["Baby_ImageNet","Papa_ImageNet", "Grandpa_ImageNet"] and ("BigGAN" in key or "ContraGAN" in key or "ReACGAN" in key):
            prefix = prefix_B
        else:
            prefix = prefix_
        
        for other in [others1, others2, others3]:
            if other != None:
                f.write(prefix + other + eval_backbone_incep + metrics_full + train_ + "\n")
                i+=1 
                f.write(prefix + other + eval_backbone_swav + metrics_full + train_+ "\n")
                i+=1 
                f.write(prefix + other + eval_backbone_swin + metrics_full + train_+ "\n")
                i+=1 

In [ ]:
#### RUN 
#### bash eval_clean_CIFAR10.sh
#### bash eval....sh

In [67]:
def accumulate_values(col, start):
    tmp = []
    for i in range(3):
        if col[start+i].value != None:
            tmp.append(col[start+i].value)
    return tmp

def fill_excel_train(col, start_row, data):
    col[start_row].value = float(data["IS"])
    col[start_row+4].value = data["FID"]
    col[start_row+8].value = data["Improved_Precision"]
    col[start_row+12].value = data["Improved_Recall"]
    col[start_row+16].value = data["Density"]
    col[start_row+20].value = data["Coverage"]
    
def fill_excel_valid(col, start_row, data):
    col[start_row].value = data["FID"]
def fill_excel_ifid(col, start_row, data):
    col[start_row].value = sum(data)/len(data)
def fill_excel_extra(col):
    pairs = [(10, 107), (42,134), (74,161)]
    for pair in pairs:
        for i in range(8):
            tmp = accumulate_values(col, pair[0]+4*i)
            if len(tmp) == 0:
                col[pair[1]+3*i].value = None
                col[pair[1]+3*i+1].value = None
            elif len(tmp) == 1:
                col[pair[1]+3*i].value = round(np.mean(tmp),2)
                col[pair[1]+3*i+1].value = "*"
            else:
                col[pair[1]+3*i].value = round(np.mean(tmp),2)
                col[pair[1]+3*i+1].value = round(np.std(tmp, ddof=0),3)

def check_SN(name):
    if name.startswith("CIFAR10-SNGAN-Diff"):
        return False
    if name.startswith("CIFAR10-SNGAN-ADA"):
        return False
    if name.startswith("CIFAR10-SNGAN-APA"):
        return False
    if name.startswith("CIFAR10-SNGAN-LeCam"):
        return False
    return True
    
def get_value(col, row):
    if col[row].value != None:
        return col[row].value
    else:
        return 0
    
def get_pickle_names(col):
    pickle_names = defaultdict(dict)
    run_name1, run_name2, run_name3 = col[6].value, None, None
    if not run_name1.startswith("CIFAR10_") and check_SN(run_name1):
        backbones = ["InceptionV3_tf", "SwAV_torch", "Swin-T_torch"]
    else:
        backbones = ["InceptionV3_tf"]

    valid = "test" if run_name1.startswith("CIFAR") else "valid"
    if col[7].value != None:
        run_name2 = col[7].value
    if col[8].value != None:
        run_name3 = col[8].value
    for i, run_name in enumerate([run_name1, run_name2, run_name3]):
        if run_name != None:
            for backbone in backbones:
                pickle_names[f"run_name{i+1}"][f"{backbone}-train"] = run_name+f"-{backbone}-train.pickle"
                if not run_name.startswith("CIFAR10_") and check_SN(run_name):
                    pickle_names[f"run_name{i+1}"][f"{backbone}-ifid"] = run_name+f"-{backbone}-train-ifid.pickle"
                    if not run_name.startswith("AFHQ_V2_uncond"):
                        pickle_names[f"run_name{i+1}"][f"{backbone}-valid"] = run_name+f"-{backbone}-{valid}.pickle"
    return pickle_names


In [75]:
# load Excel File and read informations
import openpyxl
from collections import defaultdict
import pickle
import os
import numpy as np

wb = openpyxl.load_workbook("Taxonomy_experiments_offline.xlsx")
pickles_path = "./eval_pickles/"
pickles_list = os.listdir(pickles_path)
ws_list = ["AFHQ-V2-512-uncond-tailored", "CIFAR10_tailored", "ImageNet_tailored", "Baby_ImageNet_tailored", "Papa_ImageNet_tailored", "Grandpa_ImageNet_tailored", "Efficient-AFHQ-V2-256", "Efficient-CIFAR10-100%", "Efficient-CIFAR10-30%", "Efficient-CIFAR10-10%"]
IS_start_row = 10
SwAV_start_row = 42
SwinT_start_row = 74

for ws in ws_list:
    ws = wb[ws]
    all_columns = ws.columns
    
    for idx, col in enumerate(all_columns):        
        if idx >=2 and col[1].value != None:
            pickle_names = get_pickle_names(col)
            
            for i, key in enumerate(pickle_names.keys()): 
                # i, key = run_name1,2,3
                for key, value in pickle_names[key].items():
                    # key = "backbone-train/valid/ifid", value = "~~.pickle"
                    
                    # get starting row
                    if key.startswith("Inc"):
                        start_row = IS_start_row
                    elif key.startswith("SwAV"):
                        start_row = SwAV_start_row
                    else:
                        start_row = SwinT_start_row
                        
                    with open(os.path.join(pickles_path, value), "rb") as f:
                        data = pickle.load(f)
                        if key.endswith("train"):
                            fill_excel_train(col, start_row + i, data)
                        if key.endswith("valid"):
                            fill_excel_valid(col, start_row + i + 24, data)
                        if key.endswith("ifid"):
                            fill_excel_ifid(col, start_row + i + 28, data)
    
    # fill average and stds
    all_columns = ws.columns
    fid_inc, fid_swav, fid_swin = [], [], []
    efid_inc, efid_swav, efid_swin = [], [], []
    is_inc, is_swav, is_swin = [], [], []
    ifid_inc, ifid_swav, ifid_swin = [], [], []
    
    for idx, col in enumerate(all_columns):
        if idx >= 2 and col[1].value != None:
            fill_excel_extra(col)
            
            for idx, backbone in enumerate([is_inc, is_swav, is_swin]):
                if col[107+27*idx].value != None:
                    backbone.append(col[107+27*idx].value)  
                    
            for idx, backbone in enumerate([fid_inc, fid_swav, fid_swin]):
                if col[110+27*idx].value != None:
                    backbone.append(col[110+27*idx].value)  
                    
            for idx, backbone in enumerate([efid_inc, efid_swav, efid_swin]):
                if col[125+27*idx].value != None:
                    backbone.append(col[125+27*idx].value)   
                    
            for idx, backbone in enumerate([ifid_inc, ifid_swav, ifid_swin]):
                if col[128+27*idx].value != None:
                    backbone.append(col[128+27*idx].value)            
                
    
    # fill fid rank
    all_columns = ws.columns
    for idx, col in enumerate(all_columns):
        if idx >= 2 and col[1].value != None:
            for idx, backbone in enumerate([is_inc, is_swav, is_swin]):
                if col[107+27*idx].value != None:
                    col[131+27*idx].value = sorted(backbone, reverse=True).index(col[107+27*idx].value)+1

            for idx, backbone in enumerate([fid_inc, fid_swav, fid_swin]):
                if col[110+27*idx].value != None:
                    col[132+27*idx].value = sorted(backbone).index(col[110+27*idx].value)+1
                    
            for idx, backbone in enumerate([efid_inc, efid_swav, efid_swin]):
                if col[125+27*idx].value != None:
                    col[133+27*idx].value = sorted(backbone).index(col[125+27*idx].value)+1
            
            
            for idx, backbone in enumerate([ifid_inc, ifid_swav, ifid_swin]):
                if col[128+27*idx].value != None:
                    col[133+27*idx].value = f"{col[133+27*idx].value} / {sorted(backbone).index(col[128+27*idx].value)+1}"
            
wb.save("Taxonomy_experiments_offline.xlsx")